In [ ]:
import pandas as pd
import re
import numpy as np

## Upload files and convert to dataframe using pandas to prepare for data manipulation 

There will be 3 datasets to upload:


*   bom_result.csv (contains item details such as category data)
*   ItemSoldoveraperiodDigicorResults.xls (contains quantity sold for each item)
*   customer_search.csv (website data from 2018)

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving bom_result.csv to bom_result.csv
Saving customer_search.csv to customer_search.csv
Saving ItemSoldoveraperiodDigicorResults.xls to ItemSoldoveraperiodDigicorResults.xls


In [ ]:
data = pd.read_csv('customer_search.csv')
qtySold = pd.read_excel('ItemSoldoveraperiodDigicorResults.xls')

fields = ['name', 'desc', 'Category', 'bom_type ID', 'BOM Type'] 
bom_result = pd.read_csv('bom_result.csv', usecols = fields)

## 'qtySold' Preparation

Items that start with 'BB-' are the items to focus on in this project. This code removes all items that do not have 'BB-' in their name. 

In [ ]:
qtySold = qtySold[qtySold['Item Code'].str.contains("BB-")]

In 'qtySold' and item name will be listed as BB-{Item Name}

'BB-' is stripped from the item name so it could match with other tables.

In [ ]:
qtySold['Item Code'] = qtySold['Item Code'].map(lambda x: x.lstrip('BB-'))

Sum total quantity sold for each item.

In [ ]:
qtySold = qtySold.groupby(['Item Code']).agg({'Qty Sold': [sum]})

Every time data is aggregated, Python likes to create a multi-level index. The following code simply resets the index back to a single level to avoid potential complications. 

In [ ]:
qtySold.columns = qtySold.columns.droplevel(level=0)
qtySold = qtySold.add_suffix('_sum').reset_index()

In [ ]:
qtySold = qtySold.rename(columns={"sum_sum": "Qty Sold"}) 

In [ ]:
qtySold

,Item Code,Qty Sold
0,1013S-MTR,8.0
1,1014S-WTRT,31.0
2,1019C-HTN2,13.0
3,1019P-FHN2T,3.0
4,1019P-WTR,26.0
...,...,...
140,SBA-4119S-C2N,18.0
141,SBE-820J-622,5.0
142,SBI-4129P-C2N,64.0
143,SBI-7428R-C3N,54.0


## Website data preparation

'Avg. Time on Page' is a feature that has NaN values. NaN values have been interpreted as a user clicking on a page and not having any activity and or has somehow immediately left the page. Based on this interpretation, each NaN value was changed to 0.

In [ ]:
data['Avg. Time on Page'] = data['Avg. Time on Page'].fillna(0)

In [ ]:
data['Pageviews']

0         1
1         3
2         2
3         5
4         9
         ..
47729     1
47730     1
47731     1
47732     1
47733    14
Name: Pageviews, Length: 47734, dtype: int64

Before aggregating by date: New column = avg time on page * page views
Aggregate by date and divide newcolumn/total number of page views to get an approximate inferred avg time on page.

We calculate the 'Avg. Time on Page' per item by:


*   Creating a new column that contains Avg. Time on Page * Page Views. 
*   Aggregate by Page so that all the website metrics are summed into a total for each Page
*   Divide the new column by page views so that Avg. Time on Page is now properly represents a weighted average 







In [ ]:
avgt_pv = data['Avg. Time on Page']*data['Pageviews']

In [ ]:
data['avg*pv'] = avgt_pv

In [ ]:
x = data.groupby(['Page']).agg({
    
    'Pageviews' : [sum],
    
    'Organic Searches': [sum],
    
    'New Users': [sum],
    
    'Avg. Time on Page': [sum],

    'Number of Sessions per User': [sum],

    'Users': [sum],

    'avg*pv': [sum]
  }  
)

In [ ]:
x.columns = x.columns.droplevel(level=0)
x = x.add_suffix('_sum').reset_index()

In [ ]:
x = x.set_axis(['Page', 'Page_Views', 'Organic_Searches', 'New_Users', 'Avg_Time_on_Page', 'Number_of_Sessions_per_User', 'Users', 'avg*sum'], axis=1, inplace=False)

In [ ]:
weighted_avg = x['avg*sum']/x['Page_Views']

In [ ]:
x['Weighted_Avg_Time_on_Page'] = weighted_avg

In [ ]:
data = x

## Join tables and final cleanup
Add a column to 'qtySold' table to store the 'Page' name that is associated with the item.

In [ ]:
qtySold['Page'] = "n/a"

Each page in the website data has the item name in it. 

I.E The item '1013S-MTR' will be associated with the page '/systems/AS-1013S-MTR'

To match each item from 'qtySold' with its page in the website data table, we use regex so that for each item, it will iterate through each webpage until there is a match. Once there is a match, that webpage is stored in the 'qtySold' table. 

In [ ]:
for item in qtySold['Item Code']:
  for page in data['Page']:
    x = re.search(item + '$', page)
    if x != None:
      qtySold.loc[qtySold['Item Code'] == item, 'Page'] = page

Merge bom_result and qtySold to form qtySold_bom_result using an outer join.

Since it was an outer join, we will have a bunch of irrelevant rows. We remove these by dropping NaN rows.

In [ ]:
bom_result = bom_result.rename(columns={"desc": "Item Code"})

In [ ]:
qtySold_bom_result = pd.merge(qtySold, bom_result, how = "outer", on = ['Item Code'])

In [ ]:
qtySold_bom_result.rename(columns={"Qty Sold": "Qty_Sold"}, inplace=True)
qtySold_bom_result = qtySold_bom_result.dropna(subset=['Qty_Sold'])
qtySold_bom_result['name'].fillna("n/a", inplace = True)

We were interested in using BOM ID to see investigate relationships per BOM type. 'Name' and 'Category' became irrelevent so it was dropped.

In [ ]:
qtySold_bom_result = qtySold_bom_result.drop(columns=['name', 'Category'])

Merge qtySold_bom_result with website data

In [ ]:
webdata_qtySold = pd.merge(qtySold_bom_result, data, how = "inner", on = ['Page'])

Drop rows with NaN organic searches and NaN item code

In [ ]:
webdata_qtySold = webdata_qtySold.dropna(subset=['Organic_Searches', 'Item Code'])

Items with a NaN Bom type listed as 'Other'

In [ ]:
webdata_qtySold['bom_type ID'] = webdata_qtySold['bom_type ID'].replace(np.nan, 'Other')

In [ ]:
webdata_qtySold['BOM Type'] = webdata_qtySold['BOM Type'].replace(np.nan, 'Other')

Advised to ignore pages that are missing or have '/searchGlobal?q=' in the page name.  These pages are queries for DiGiCOR's embedded search feature.

In [ ]:
index = 0
for page in webdata_qtySold['Page']:
    x = re.search('^/searchGlobal', page)
    if x != None:
      webdata_qtySold = webdata_qtySold.drop([index])
    
    index += 1

Some cleaning up:
*   Removing duplicates
*   Dropping unwanted columns



In [ ]:
webdata_qtySold = webdata_qtySold.drop_duplicates(subset=['Item Code'])

In [ ]:
webdata_qtySold = webdata_qtySold.drop(columns=['Avg_Time_on_Page', 'avg*sum'])

In [ ]:
webdata_qtySold = webdata_qtySold.reset_index()

In [ ]:
webdata_qtySold

,index,Item Code,Qty_Sold,Page,bom_type ID,BOM Type,Page_Views,Organic_Searches,New_Users,Number_of_Sessions_per_User,Users,Weighted_Avg_Time_on_Page
0,0,1013S-MTR,8.0,/systems/AS-1013S-MTR,2,BB,96,15,17,27.500000,80,175.812500
1,2,1014S-WTRT,31.0,/systems/AS-1014S-WTRT,2,BB,184,40,51,65.666667,142,141.908514
2,3,1019C-HTN2,13.0,/systems/superServer-1019C-HTN2,2,BB,5,2,4,4.000000,5,10.800000
3,5,1019P-FHN2T,3.0,/systems/SuperServer-1019P-FHN2T,2,BB,101,13,20,28.500000,87,78.643564
4,6,1019P-WTR,26.0,/systems/superServer-1019P-WTR,2,BB,1,1,1,1.000000,1,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
85,124,MBI-6118G-T41X,1.0,/systems/MBI-6118G-T41X,5,Node,1,0,0,0.000000,1,30.000000
86,125,R1208WFTYS,9.0,/systems/intel-R1208WFTYS,2,BB,5,2,5,3.000000,5,0.000000
87,127,R2224WFTZSR,10.0,/systems/Intel-R2224WFTZSR,2,BB,13,1,1,1.000000,10,80.461538
88,128,RS500A-E9-RS4,8.0,/systems/RS500A-E9-RS4,2,BB,117,11,14,19.000000,85,88.733211


Final dataset to use for modelling called 'webdata_qtySold'.

In [ ]:
webdata_qtySold.to_csv("webdata_qtySold.csv")
files.download('webdata_qtySold.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Missing data
This code was used to compile a dataset that contained:


*   Items with a missing webpage
*   Items with a missing category, bom type ID





In [ ]:
missing_page_items = qtySold_bom_result.loc[qtySold_bom_result['Page'] == "n/a"]

In [ ]:
missing_bom_result = qtySold_bom_result.loc[qtySold_bom_result['Page'] != "n/a"]

In [ ]:
missing_page_items.to_csv("Items with missing pages.csv")
missing_bom_result.to_csv("Items with missing category, name, bom_type ID and bom type.csv")